In [4]:
!pip install phidata reportlab python-dotenv


   ---------------------------------------- 0.0/2.0 MB ? eta -:--:--
   ----- ---------------------------------- 0.3/2.0 MB ? eta -:--:--
   ---------------- ----------------------- 0.8/2.0 MB 2.0 MB/s eta 0:00:01
   ---------------- ----------------------- 0.8/2.0 MB 2.0 MB/s eta 0:00:01
   --------------------- ------------------ 1.0/2.0 MB 1.5 MB/s eta 0:00:01
   -------------------------- ------------- 1.3/2.0 MB 1.1 MB/s eta 0:00:01
   -------------------------- ------------- 1.3/2.0 MB 1.1 MB/s eta 0:00:01
   -------------------------------- ------- 1.6/2.0 MB 1.1 MB/s eta 0:00:01
   ---------------------------------------- 2.0/2.0 MB 1.1 MB/s eta 0:00:00



[notice] A new release of pip is available: 24.3.1 -> 25.3
[notice] To update, run: python.exe -m pip install --upgrade pip


In [5]:
from dotenv import load_dotenv
import os

load_dotenv()  # loads .env file

# Verify (optional, safe check)
assert os.getenv("GROQ_API_KEY") is not None, "GROQ_API_KEY not found in .env"


In [13]:
from phi.agent import Agent
from phi.model.groq import Groq

model = Groq(
    id="openai/gpt-oss-120b"   # <-- model name goes here
)


In [19]:
health_agent = Agent(
    model=model,
    instructions="""
    You are a health expert.
    List the most healthy daily routines based on science.
    """,
    markdown=True
)

summary_agent = Agent(
    model=model,
    instructions="Summarize the content clearly.",
    markdown=True
)

top3_agent = Agent(
    model=model,
    instructions="Extract only the top 3 most important points.",
    markdown=True
)
email_agent = Agent(
    model=model,
    instructions="""
    You are a professional email writer.

    Using the provided content:
    - Write a clear, friendly, and professional email
    - Include a subject line
    - Add a greeting
    - Present the content concisely
    - End with a polite closing

    Do NOT add new information.
    """,
    markdown=True
)



In [20]:
# Step 1: Agent 1 → Healthy routines
health_routines = health_agent.run(
    "Tell me the top most healthy daily routines."
).content

# Step 2: Agent 2 → Summarize
summary_1 = summary_agent.run(health_routines).content

# Step 3: Agent 3 → Extract Top 3
top_3 = top3_agent.run(summary_1).content

# Step 4: Agent 2 → Final summary
final_summary = summary_agent.run(top_3).content

final_summary


'## Quick Summary: 3️⃣ Daily Routines That Most Impact Health  \n\n| # | What to Prioritize | Why It’s Crucial (Science‑Backed) | Simple Action Step |\n|---|-------------------|----------------------------------|--------------------|\n| **1️⃣** | **Get Quality Sleep – 7‑9\u202fh/night** | Lowers risk of heart disease, obesity, depression, and premature death; deep sleep sharpens memory and strengthens immunity. | Set a regular bedtime, dim lights 30\u202fmin before sleep, and keep the room cool (≈\u202f18‑20\u202f°C). |\n| **2️⃣** | **Move Every Day** | 150\u202fmin of moderate (or 75\u202fmin vigorous) cardio + strength work twice a week cuts all‑cause mortality by ~30\u202f%; short micro‑breaks improve glucose control and mood. | Aim for 30\u202fmin of walking/jogging/biking daily, add 5‑10\u202fmin of body‑weight exercises 2‑3×/week, and stand/stretch each hour. |\n| **3️⃣** | **Stay Hydrated** | Losing just 1‑2\u202f% of body water hampers cognition, mood, and performance; water su

ValueError: 
paragraph text '<para>| **1** | **High\u2011Quality Sleep (7\u20119 h/night)** | \u2022 Removes metabolic waste from the brain<br>\u2022 Regulates hunger hormones (leptin & ghrelin)<br>\u2022 Lowers risk of hypertension, type 2 diabetes, obesity, and mental\u2011health disorders *(AASM 2023)* | Set a fixed bedtime & wake\u2011time; dim lights 30 min before bed to cue melatonin. |</para>' caused exception paraparser: syntax error: No content allowed in br tag

In [18]:
from IPython.display import display, Markdown

# Step 1: Agent 1 → Healthy routines
health_routines = health_agent.run(
    "Tell me the top most healthy daily routines."
).content

display(Markdown("## 🥗 Agent 1: Healthy Daily Routines"))
display(Markdown(health_routines))


# Step 2: Agent 2 → First Summary
summary_1 = summary_agent.run(health_routines).content

display(Markdown("## ✍️ Agent 2: Summary of Healthy Routines"))
display(Markdown(summary_1))


# Step 3: Agent 3 → Extract Top 3
top_3 = top3_agent.run(summary_1).content

display(Markdown("## ⭐ Agent 3: Top 3 Key Points"))
display(Markdown(top_3))


# Step 4: Agent 2 → Final Summary
final_summary = summary_agent.run(top_3).content

display(Markdown("## 📌 Agent 2: Final Refined Summary"))
display(Markdown(final_summary))

# Step 5: Agent 4 → Generate Email
email_content = email_agent.run(final_summary).content

display(Markdown("## 📧 Agent 4: Email Generated from Final Summary"))
display(Markdown(email_content))



## 🥗 Agent 1: Healthy Daily Routines

## The 10 Science‑Backed Daily Routines That Boost Health  

| # | Routine | Why it matters (science) | How to implement |
|---|---------|--------------------------|------------------|
| **1** | **Prioritize 7–9 h of quality sleep** | Sleep regulates hormones (leptin, ghrelin), consolidates memory, and clears metabolic waste from the brain (glymphatic system). Chronic short sleep raises risk of obesity, diabetes, cardiovascular disease, and mood disorders. <br>*(Refs: Walker 2020; Xie et al., 2013)* | • Keep a consistent bedtime/wake‑time (even on weekends). <br>• Dim lights 30 min before bed; avoid screens or use blue‑light filters. <br>• Cool, dark, quiet bedroom (≈18 °C, blackout curtains, white‑noise if needed). |
| **2** | **Move your body – at least 150 min of moderate aerobic activity + strength work** | Regular aerobic exercise improves VO₂ max, insulin sensitivity, and endothelial function. Resistance training preserves muscle mass and bone density, reducing frailty and mortality risk. <br>*(Refs: WHO 2020 guidelines; ACSM 2022)* | • 30 min brisk walk, jog, cycling, or swimming 5 × week. <br>• 2 × week full‑body strength session (bodyweight, bands, or weights). <br>• Break up sitting: stand or walk 2–3 min every 30 min. |
| **3** | **Eat a nutrient‑dense, balanced diet** | Diets rich in plant foods, healthy fats, and lean protein lower inflammation, improve gut microbiome diversity, and protect against chronic disease. The Mediterranean and DASH patterns are repeatedly linked to lower mortality. <br>*(Refs: Estruch et al., 2018; Sacks et al., 2001)* | • Fill half your plate with vegetables/fruits, a quarter with whole grains, and a quarter with protein (fish, legumes, nuts, poultry). <br>• Use extra‑virgin olive oil or nuts for healthy fats. <br>• Limit added sugars, refined grains, and processed meats. |
| **4** | **Stay hydrated (≈2–3 L water/day for most adults)** | Even mild dehydration impairs cognition, mood, and physical performance; adequate fluid balance supports kidney function and cardiovascular health. <br>*(Refs: Armstrong & Sullivan, 2019)* | • Sip water throughout the day; a good rule‑of‑thumb is a glass (≈250 ml) every hour. <br>• Adjust for activity, heat, and body size. <br>• Use a reusable bottle and set reminders if needed. |
| **5** | **Get 10–30 min of natural sunlight** | Sunlight triggers vitamin D synthesis and regulates circadian rhythms, which influence sleep quality, hormone balance, and mood. <br>*(Refs: Holick 2021; Czeisler et al., 2022)* | • Walk or sit outdoors in the morning (or midday) when possible. <br>• Aim for 15 min of skin exposure (face, arms) on clear days; supplement vitamin D if you live far from the equator or have limited sun. |
| **6** | **Practice stress‑management techniques** | Chronic stress elevates cortisol, promotes visceral fat accumulation, and impairs immune function. Mind‑body practices lower stress biomarkers and improve mental health. <br>*(Refs: Cohen et al., 2020; Goyal et al., 2014)* | • 5–10 min of deep‑breathing, progressive muscle relaxation, or meditation daily. <br>• Consider yoga, tai‑chi, or a brief walk in nature as active stress relief. |
| **7** | **Cultivate social connection** | Strong relationships predict lower mortality, better immune response, and reduced risk of depression. Social isolation is a comparable risk factor to smoking. <br>*(Refs: Holt‑Lunstad et al., 2015)* | • Call, video‑chat, or meet a friend/family member at least once a day. <br>• Join a community group, club, or volunteer activity that aligns with your interests. |
| **8** | **Limit alcohol & avoid tobacco** | Excess alcohol raises blood pressure, liver disease risk, and certain cancers. Smoking is the leading preventable cause of death worldwide. <br>*(Refs: WHO 2022; CDC 2023)* | • If you drink, keep it ≤ 1 drink/day for women, ≤ 2 drinks/day for men. <br>• Use nicotine‑replacement or counseling programs to quit smoking. |
| **9** | **Practice good oral hygiene** | Periodontal disease is linked to systemic inflammation, heart disease, and poor glycemic control. Brushing and flossing reduce bacterial load. <br>*(Refs: Petersen & Tonetti, 2021)* | • Brush twice daily with fluoride toothpaste (2 min each). <br>• Floss or use interdental cleaners once daily. <br>• Replace toothbrush every 3 months; schedule dental check‑ups twice a year. |
| **10** | **Schedule preventive health checks** | Early detection of hypertension, cholesterol abnormalities, cancers, and vision/hearing loss enables timely intervention and reduces morbidity. <br>*(Refs: USPSTF 2023)* | • Annual primary‑care visit (blood pressure, BMI, labs). <br>• Age‑appropriate screenings (e.g., colonoscopy ≥ 45 y, mammogram ≥ 40 y, eye exam ≥ 60 y). <br>• Keep vaccination schedule up to date (flu, COVID‑19, shingles, etc.). |

---

### Quick “Starter” Checklist (5‑minute daily audit)

- [ ] **Did I sleep 7‑9 h?**  
- [ ] **Did I move for at least 30 min?**  
- [ ] **Did I drink enough water?**  
- [ ] **Did I eat ≥ 5 servings of fruits/vegetables?**  
- [ ] **Did I get sunlight or vitamin D?**  
- [ ] **Did I practice a stress‑relief technique?**  
- [ ] **Did I connect with someone?**  

If you’re consistently “yes” on most items, you’re on a strong health trajectory. Adjust one or two habits at a time rather than trying to overhaul everything at once—behavior change is most sustainable when incremental.

---

#### References (selected)

1. Walker, M. (2020). *Why We Sleep*. Scribner.  
2. Xie, L. et al. (2013). “Sleep drives metabolite clearance from the adult brain.” *Science*, 342, 373‑377.  
3. World Health Organization. (2020). *Physical activity guidelines for adults*.  
4. Estruch, R. et al. (2018). “Primary prevention of cardiovascular disease with a Mediterranean diet.” *NEJM*, 378, 1279‑1290.  
5. Holick, M.F. (2021). “Vitamin D deficiency.” *NEJM*, 385, 373‑384.  
6. Goyal, M. et al. (2014). “Meditation programs for psychological stress and well‑being.” *JAMA Intern Med*, 174, 134‑143.  
7. Holt‑Lunstad, J. et al. (2015). “Loneliness and social isolation as risk factors for mortality.” *Perspect Psychol Sci*, 10, 227‑237.  
8. Cohen, S. et al. (2020). “Stress, inflammation, and disease.” *Nat Rev Immunol*, 20, 273‑285.  
9. Petersen, P.E. & Tonetti, M.S. (2021). “Periodontitis and systemic health.” *J Clin Periodontol*, 48, 1‑20.  
10. U.S. Preventive Services Task Force (USPSTF). (2023). *Screening recommendations*.  

> *Tip:* Bookmark this page or print the checklist and place it somewhere visible (e.g., bathroom mirror) to reinforce these habits each day. Consistency is the key ingredient for lasting health.

## ✍️ Agent 2: Summary of Healthy Routines

## 10 Science‑Backed Daily Routines for Better Health  

| # | Core Habit | What the Science Says | Simple Ways to Do It |
|---|------------|----------------------|----------------------|
| **1** | **7‑9 h quality sleep** | Regulates hormones (leptin ↔ ghrelin), consolidates memory, clears brain waste (glymphatic system). Short sleep ↑ obesity, diabetes, CVD, mood disorders. | • Same bedtime & wake‑time daily (even weekends) <br>• Dim lights 30 min before bed; limit screens or use blue‑light filter <br>• Cool, dark, quiet room (~18 °C) |
| **2** | **150 min moderate aerobic + strength** | Boosts VO₂ max, insulin sensitivity, vascular health. Resistance training preserves muscle & bone, cuts frailty & mortality risk. | • 30 min brisk walk/jog/cycle/swim × 5 weeks <br>• 2 full‑body strength sessions/week (bodyweight, bands, weights) <br>• Stand or walk 2‑3 min every 30 min of sitting |
| **3** | **Nutrient‑dense balanced diet** | Plant‑rich, healthy‑fat, lean‑protein meals ↓ inflammation, improve gut microbiome, lower chronic‑disease risk (Mediterranean/DASH). | • Plate: ½ veg/fruit, ¼ whole‑grains, ¼ protein (fish, legumes, nuts, poultry) <br>• Use extra‑virgin olive oil or nuts for fats <br>• Cut added sugars, refined grains, processed meats |
| **4** | **2–3 L water/day** | Even mild dehydration hurts cognition, mood & physical performance; proper hydration supports kidneys & heart. | • Sip ~250 ml (1 cup) each hour <br>• Adjust for exercise, heat, body size <br>• Keep a reusable bottle & set reminders |
| **5** | **10–30 min natural sunlight** | Triggers vitamin D synthesis & syncs circadian clock → better sleep, hormones & mood. | • Morning or midday walk/sit outdoors <br>• 15 min skin exposure (face/arms) on clear days <br>• Supplement vitamin D if sun exposure is limited |
| **6** | **Stress‑management (5‑10 min)** | Chronic cortisol ↑ visceral fat, impairs immunity. Mind‑body practices lower stress biomarkers & boost mental health. | • Deep‑breathing, progressive muscle relaxation, or meditation <br>• Yoga, tai‑chi, or a brief nature walk as active relief |
| **7** | **Social connection** | Strong relationships predict lower mortality, better immune response & less depression; isolation ≈ smoking risk. | • Call, video‑chat, or meet a friend/family daily <br>• Join a club, class, or volunteer group you enjoy |
| **8** | **Limit alcohol & avoid tobacco** | Excess alcohol → hypertension, liver disease, cancers. Smoking is the top preventable death cause. | • ≤ 1 drink/day (women) or ≤ 2 drinks/day (men) <br>• Use nicotine‑replacement, counseling, or quit‑lines to stop smoking |
| **9** | **Good oral hygiene** | Periodontal disease links to systemic inflammation, heart disease & poor glucose control. | • Brush 2 min with fluoride toothpaste twice daily <br>• Floss or use interdental cleaners once daily <br>• Replace toothbrush every 3 months; dental check‑up twice a year |
| **10** | **Preventive health checks** | Early detection of hypertension, cholesterol, cancers, vision/hearing loss cuts morbidity & mortality. | • Annual primary‑care visit (BP, BMI, labs) <br>• Age‑appropriate screenings (e.g., colonoscopy ≥ 45 y, mammogram ≥ 40 y) <br>• Keep vaccinations up to date (flu, COVID‑19, shingles, etc.) |

---

### Quick “5‑Minute Daily Audit”

| ✅ | Question |
|---|----------|
| 1 | Did I sleep 7‑9 h? |
| 2 | Did I move for ≥ 30 min (walk, jog, bike, etc.)? |
| 3 | Did I drink enough water? |
| 4 | Did I eat ≥ 5 servings of fruits/vegetables? |
| 5 | Did I get sunlight or vitamin D? |
| 6 | Did I practice a stress‑relief technique? |
| 7 | Did I connect with someone? |

*If most answers are “yes,” you’re on a strong health trajectory. Focus on changing one or two habits at a time—small, consistent tweaks are the most sustainable.*

---

### Take‑away Tips

1. **Start small.** Pick the habit that feels easiest to add this week and build from there.  
2. **Use cues.** Pair a new habit with an existing routine (e.g., “after brushing teeth → 5‑min meditation”).  
3. **Track progress.** A simple checklist on your phone or bathroom mirror reinforces consistency.  
4. **Stay flexible.** Adjust water, sunlight, and exercise amounts based on season, activity level, and personal needs.  

> **Bottom line:** Consistency across these ten evidence‑based habits creates a synergistic effect—each supports the others and together they dramatically boost overall health and longevity.  

---  

#### Selected References  

1. Walker, M. *Why We Sleep* (2020).  
2. Xie et al. “Sleep drives metabolite clearance from the adult brain.” *Science* 342 (2013).  
3. WHO. *Physical activity guidelines for adults* (2020).  
4. Estruch et al. “Primary prevention of cardiovascular disease with a Mediterranean diet.” *NEJM* 378 (2018).  
5. Holick, M.F. “Vitamin D deficiency.” *NEJM* 385 (2021).  
6. Goyal et al. “Meditation programs for psychological stress and well‑being.” *JAMA Intern Med* 174 (2014).  
7. Holt‑Lunstad et al. “Loneliness and social isolation as risk factors for mortality.” *Perspect Psychol Sci* 10 (2015).  
8. Cohen et al. “Stress, inflammation, and disease.” *Nat Rev Immunol* 20 (2020).  
9. Petersen & Tonetti. “Periodontitis and systemic health.” *J Clin Periodontol* 48 (2021).  
10. USPSTF. *Screening recommendations* (2023).  

## ⭐ Agent 3: Top 3 Key Points

**Top 3 Most Important Daily Health Habits**

| # | Core Habit | Why It Matters (Science) | Easy Way to Start |
|---|------------|--------------------------|-------------------|
| **1** | **7‑9 h quality sleep** | Regulates hunger hormones (leptin ↔ ghrelin), clears brain waste via the glymphatic system, and supports memory and mood. Short sleep ↑ risk of obesity, diabetes, cardiovascular disease, and depression. | • Go to bed and wake up at the same time every day (yes, weekends).<br>• Dim lights 30 min before bed & use a blue‑light filter on screens.<br>• Keep the bedroom cool, dark, and quiet (~18 °C). |
| **2** | **150 min moderate aerobic + strength training each week** | Boosts VO₂ max, insulin sensitivity, and vascular health; resistance work preserves muscle‑bone mass, cuts frailty and mortality risk. | • Walk, jog, bike, or swim ≈ 30 min × 5 days.<br>• Add 2 full‑body strength sessions/week (bodyweight, bands, or dumbbells).<br>• Stand or move 2‑3 min every 30 min of sitting. |
| **3** | **Nutrient‑dense balanced diet** | Plant‑rich, healthy‑fat, lean‑protein meals lower inflammation, improve the gut microbiome, and reduce chronic‑disease risk (e.g., Mediterranean/DASH patterns). | • Fill half your plate with vegetables/fruits, a quarter with whole grains, and a quarter with lean protein (fish, legumes, poultry).<br>• Use extra‑virgin olive oil or nuts for fats.<br>• Cut added sugars, refined grains, and processed meats. |

## 📌 Agent 2: Final Refined Summary

## Quick Summary of the 3 Most Important Daily Health Habits  

| # | Core Habit | Why It Matters (Science) | Easy Way to Start |
|---|------------|--------------------------|-------------------|
| **1** | **7‑9 h quality sleep** | • Balances hunger hormones (leptin & ghrelin) <br>• Clears brain waste via the glymphatic system <br>• Supports memory, mood, and reduces risk of obesity, diabetes, heart disease, and depression | • Keep a consistent bedtime & wake‑time (even on weekends) <br>• Dim lights 30 min before bed; use blue‑light filters on devices <br>• Make the bedroom cool, dark, and quiet (~18 °C) |
| **2** | **150 min moderate aerobic + strength training per week** | • Improves VO₂ max, insulin sensitivity, and vascular health <br>• Resistance work preserves muscle‑bone mass, lowers frailty & mortality risk | • Walk/jog/bike/swim ≈ 30 min × 5 days <br>• Add 2 full‑body strength sessions (bodyweight, bands, or dumbbells) <br>• Stand or move 2‑3 min every 30 min of sitting |
| **3** | **Nutrient‑dense balanced diet** | • Plant‑rich, healthy‑fat, lean‑protein meals cut inflammation, support gut microbiome, and reduce chronic‑disease risk (e.g., Mediterranean/DASH) | • Fill half the plate with vegetables/fruits, a quarter with whole grains, a quarter with lean protein (fish, legumes, poultry) <br>• Use extra‑virgin olive oil or nuts for healthy fats <br>• Limit added sugars, refined grains, and processed meats |

### Bottom Line
- **Sleep:** Consistency, darkness, cool environment.  
- **Movement:** Daily moderate cardio + 2 weekly strength sessions; break up sitting.  
- **Food:** Half‑plate veggies/fruits, quarter whole grains, quarter lean protein; choose healthy fats, cut processed sugars/meats.  

Adopting these three habits together creates a strong foundation for long‑term health and disease prevention.

In [21]:
from IPython.display import display, Markdown

# Step 1: Agent 1 → Healthy routines
health_routines = health_agent.run(
    "Tell me the top most healthy daily routines."
).content

display(Markdown("## 🥗 Agent 1: Healthy Daily Routines"))
display(Markdown(health_routines))


# Step 2: Agent 2 → First Summary
summary_1 = summary_agent.run(health_routines).content

display(Markdown("## ✍️ Agent 2: Summary of Healthy Routines"))
display(Markdown(summary_1))


# Step 3: Agent 3 → Extract Top 3
top_3 = top3_agent.run(summary_1).content

display(Markdown("## ⭐ Agent 3: Top 3 Key Points"))
display(Markdown(top_3))


# Step 4: Agent 2 → Final Summary
final_summary = summary_agent.run(top_3).content

display(Markdown("## 📌 Agent 2: Final Refined Summary"))
display(Markdown(final_summary))

# Step 5: Agent 4 → Generate Email
email_content = email_agent.run(final_summary).content

display(Markdown("## 📧 Agent 4: Email Generated from Final Summary"))
display(Markdown(email_content))



## 🥗 Agent 1: Healthy Daily Routines

## Top Science‑Backed Daily Routines for Optimal Health  

Below is a concise, evidence‑based checklist of daily habits that together form the foundation of a healthy lifestyle. Each routine is supported by peer‑reviewed research and major public‑health guidelines (e.g., WHO, CDC, American Heart Association).

| Category | Routine | Why It Matters (Key Findings) | How to Implement |
|----------|---------|------------------------------|------------------|
| **Sleep** | **7‑9 hours of quality sleep** | Adequate sleep improves cognition, immune function, hormone balance, and reduces risk of obesity, cardiovascular disease, and depression. (Sleep Med Rev 2022) | • Keep a consistent bedtime/wake‑time (even weekends).<br>• Dark, cool bedroom (≈18‑20 °C).<br>• Limit screens 30 min before bed; use “night mode”. |
| **Hydration** | **2–3 L of water (≈8–10 cups) per day** | Even mild dehydration impairs mood, attention, and physical performance. Long‑term adequate intake supports kidney health and metabolic regulation. (Nutrients 2021) | • Carry a reusable bottle.<br>• Drink a glass of water first thing, with each meal, and after exercise.<br>• Adjust for climate, activity, and body size. |
| **Nutrition – Whole Foods** | **Balanced meals rich in vegetables, fruits, whole grains, lean protein, and healthy fats** | Diets high in plant‑based foods and low in ultra‑processed items lower risk of heart disease, type‑2 diabetes, certain cancers, and promote a healthy gut microbiome. (Lancet 2023) | • Fill half your plate with non‑starchy veg.<br>• Choose whole grains (e.g., oats, quinoa).<br>• Include a source of protein (fish, legumes, poultry, tofu).<br>• Add nuts, seeds, avocado or olive oil for healthy fats. |
| **Meal Timing** | **Regular eating schedule + limited late‑night snacking** | Consistent meal timing stabilizes blood glucose and circadian rhythms, reducing metabolic disease risk. (Cell Metab 2020) | • Eat breakfast within 1 h of waking.<br>• Aim for 3 main meals (and 1–2 small snacks if needed).<br>• Finish the last meal ≥2–3 h before bedtime. |
| **Physical Activity** | **≥150 min moderate aerobic + 2 × strength sessions per week** (≈30 min most days) | Regular cardio improves heart health, insulin sensitivity, and mood; resistance training preserves muscle mass and bone density. (JAMA 2022) | • Walk, jog, bike, swim, or dance → 30 min brisk walk most days.<br>• Body‑weight, resistance‑band, or weight‑lifting circuits 2×/week.<br>• Break up sedentary time: stand or move for 2‑min every 30 min. |
| **Sunlight & Vitamin D** | **10–30 min of outdoor light exposure daily** (depending on skin tone, latitude) | Sunlight regulates circadian rhythm, boosts mood (via serotonin), and drives Vitamin D synthesis essential for bone and immune health. (J Clin Endocrinol Metab 2021) | • Take a walk, sip coffee on a balcony, or sit near a window in the morning.<br>• Use sunscreen after ~10 min if exposure is intense. |
| **Stress Management** | **5–10 min of mindfulness/relaxation** | Mind‑body practices lower cortisol, blood pressure, and inflammation; improve focus and emotional resilience. (Psychol Bull 2020) | • Guided meditation, deep‑breathing, or progressive muscle relaxation.<br>• Apps (e.g., Insight Timer, Headspace) or simple “box breathing” (4‑4‑4‑4). |
| **Social Connection** | **Meaningful interaction with at least one person** | Strong social ties are linked to a 30‑50 % lower risk of premature death, better immune response, and lower rates of depression. (PLoS Med 2021) | • Call, video‑chat, or meet a friend/family member.<br>• Participate in community groups, clubs, or volunteering. |
| **Mental Stimulation** | **30 min of cognitively challenging activity** | Engaging the brain (learning, puzzles, reading) supports neuroplasticity and may delay cognitive decline. (Neurobiol Aging 2022) | • Read a book, learn a language, play an instrument, or solve puzzles. |
| **Personal Hygiene** | **Daily oral care + regular hand washing** | Good oral hygiene reduces cardiovascular disease risk; hand hygiene limits infection spread. (Circulation 2020) | • Brush/floss twice daily; replace toothbrush every 3 months.<br>• Wash hands with soap for ≥20 s after restroom use, before meals, and after public contact. |
| **Screen Time Balance** | **≤2 h recreational screen time (excluding work)** | Excessive screen time is associated with eye strain, sleep disruption, and sedentary behavior. (BMJ 2023) | • Set device‑free zones (e.g., bedroom, meals).<br>• Use “focus” timers or blue‑light filters in the evening. |
| **Posture & Mobility** | **Micro‑stretching & posture checks every hour** | Prolonged static posture contributes to musculoskeletal pain and reduced circulation. (Spine 2021) | • Stand, roll shoulders, stretch hamstrings/hip flexors.<br>• Use a lumbar‑support pillow or sit‑stand desk if possible. |
| **Gratitude/Positive Outlook** | **3‑5 minutes journaling or expressing gratitude** | Positive affect correlates with lower inflammation, better sleep, and increased longevity. (Health Psychol 2020) | • Write 3 things you’re grateful for each night.<br>• Share a positive moment with someone. |

---

### How to Turn This List Into a Sustainable Routine  

1. **Start Small** – Pick **two** habits to adopt this week (e.g., consistent sleep schedule + 10‑min morning walk).  
2. **Stack Habits** – Pair a new habit with an existing one (e.g., drink a glass of water right after brushing teeth).  
3. **Track Progress** – Use a simple habit tracker or phone app; visual streaks reinforce consistency.  
4. **Adjust Gradually** – Once a habit feels automatic, add another. Aim for a fully rounded day within 4–6 weeks.  
5. **Listen to Your Body** – Modify intensity, timing, or food choices based on energy levels, medical conditions, and personal preferences.

---

#### Quick “Day‑in‑the‑Life” Blueprint (≈30‑minute read)

| Time | Action |
|------|--------|
| **06:30 – 07:00** | Wake, drink a glass of water, brief sunlight exposure (walk or balcony). |
| **07:00 – 07:30** | Light mobility stretch + 5‑min mindfulness. |
| **07:30 – 08:00** | Balanced breakfast (e.g., oatmeal + berries + Greek yogurt). |
| **08:00 – 12:00** | Work/activities – stand/walk 2 min each hour; sip water. |
| **12:00 – 12:30** | Lunch (vegetable‑rich plate + protein) + short walk outdoors. |
| **12:30 – 13:00** | 5‑min gratitude journal + brief breathing exercise. |
| **13:00 – 17:00** | Continue work; incorporate another 2‑min movement break each hour. |
| **17:30 – 18:15** | 30‑min moderate cardio (brisk walk, bike, swim). |
| **18:30 – 19:00** | Dinner (lean protein, whole grain, veg) + family/social interaction. |
| **19:30 – 20:00** | Light reading / puzzle (mental stimulation). |
| **20:00 – 20:30** | Oral hygiene + wind‑down screen‑free routine (dim lights, calming music). |
| **21:00** | Bedtime – aim for 7‑9 h sleep. |

*Feel free to shift times to match your personal schedule; the core elements remain the same.*

---

### References (selected)

1. **Sleep** – *Sleep Medicine Reviews*, 2022.  
2. **Hydration** – *Nutrients*, 2021.  
3. **Dietary Patterns** – *The Lancet*, 2023.  
4. **Meal Timing** – *Cell Metabolism*, 2020.  
5. **Physical Activity** – *JAMA*, 2022.  
6. **Sunlight & Vitamin D** – *Journal of Clinical Endocrinology & Metabolism*, 2021.  
7. **Mindfulness** – *Psychological Bulletin*, 2020.  
8. **Social Ties** – *PLoS Medicine*, 2021.  
9. **Cognitive Stimulation** – *Neurobiology of Aging*, 2022.  
10. **Oral Health & Cardiovascular Risk** – *Circulation*, 2020.  
11. **Screen Time** – *BMJ*, 2023.  
12. **Posture & Mobility** – *Spine*, 2021.  
13. **Gratitude & Longevity** – *Health Psychology*, 2020.  

*(Full citation list available on request.)*  

---

**Bottom line:** Incorporating these evidence‑based routines—sleep, hydration, whole‑food nutrition, regular movement, sunlight, stress reduction, social connection, mental challenge, hygiene, and mindful screen use—creates a synergistic “health stack” that dramatically improves physical, mental, and emotional wellbeing. Start small, be consistent, and watch the benefits compound over weeks and months. 🌿

## ✍️ Agent 2: Summary of Healthy Routines

## Quick‑Reference Summary: Science‑Backed Daily Routines for Optimal Health  

| **Category** | **Core Habit** | **Key Benefits (Evidence)** | **How to Make It Stick** |
|--------------|----------------|-----------------------------|---------------------------|
| **Sleep** | 7‑9 h of quality sleep each night | Better cognition, immune function, hormone balance; lowers obesity, CVD & depression risk *(Sleep Med Rev 2022)* | • Same bedtime/wake‑time daily (incl. weekends) <br>• Cool, dark room (≈18‑20 °C) <br>• No screens 30 min before bed (night mode) |
| **Hydration** | 2‑3 L water (≈8‑10 cups) | Prevents mood & performance drops; supports kidney health & metabolism *(Nutrients 2021)* | • Carry a reusable bottle <br>• Drink a glass right after waking, at meals & post‑exercise <br>• Adjust for climate, activity & size |
| **Whole‑Food Nutrition** | Balanced plates: ½ veg, whole grains, lean protein, healthy fats | Lowers heart disease, type‑2 diabetes, some cancers; nourishes gut microbiome *(Lancet 2023)* | • Fill half plate with non‑starchy veg <br>• Choose oats, quinoa, brown rice <br>• Add fish, legumes, poultry or tofu <br>• Top with nuts, seeds, avocado or olive oil |
| **Meal Timing** | Regular meals; finish dinner ≥2‑3 h before bed | Stabilises blood glucose & circadian rhythm; cuts metabolic disease risk *(Cell Metab 2020)* | • Breakfast within 1 h of waking <br>• 3 main meals (+ 1‑2 small snacks if needed) <br>• No late‑night snacking |
| **Physical Activity** | ≥150 min moderate cardio + 2 strength sessions weekly (≈30 min most days) | Improves heart health, insulin sensitivity, mood; preserves muscle & bone *(JAMA 2022)* | • Brisk walk, jog, bike, swim, dance 30 min most days <br>• Body‑weight, bands or weights 2×/wk <br>• Stand/move 2 min every 30 min of sitting |
| **Sunlight / Vitamin D** | 10‑30 min outdoor light daily (skin‑type & latitude dependent) | Sets circadian clock, boosts serotonin‑driven mood, produces Vitamin D for bone & immunity *(J Clin Endocrinol Metab 2021)* | • Morning walk, coffee on balcony, or sit by a sunny window <br>• Apply sunscreen after ~10 min if UV is strong |
| **Stress Management** | 5‑10 min mindfulness/relaxation | Lowers cortisol, blood pressure & inflammation; sharpens focus *(Psychol Bull 2020)* | • Guided meditation, deep‑breathing, box‑breathing (4‑4‑4‑4) <br>• Use apps (Insight Timer, Headspace) |
| **Social Connection** | Meaningful interaction with ≥1 person daily | 30‑50 % lower premature‑death risk; better immune response & mood *(PLoS Med 2021)* | • Call, video‑chat, meet a friend/family <br>• Join clubs, volunteer, community groups |
| **Mental Stimulation** | 30 min cognitively challenging activity | Enhances neuroplasticity; may delay cognitive decline *(Neurobiol Aging 2022)* | • Read, learn a language, play an instrument, do puzzles |
| **Personal Hygiene** | Daily oral care + regular hand washing | Good oral health ↓ cardiovascular risk; hand hygiene ↓ infections *(Circulation 2020)* | • Brush & floss twice daily; change toothbrush qtrly <br>• Soap wash ≥20 s after restroom, before meals, after public contact |
| **Screen‑Time Balance** | ≤2 h recreational screen time (excl. work) | Reduces eye strain, sleep disruption & sedentary time *(BMJ 2023)* | • Device‑free zones (bedroom, meals) <br>• Use focus timers or blue‑light filters in evenings |
| **Posture & Mobility** | Micro‑stretch & posture check each hour | Prevents musculoskeletal pain & circulation drops *(Spine 2021)* | • Stand, roll shoulders, stretch hamstrings/hip flexors <br>• Use lumbar support or sit‑stand desk if possible |
| **Gratitude / Positive Outlook** | 3‑5 min journaling or gratitude sharing | Lowers inflammation, improves sleep & longevity *(Health Psychol 2020)* | • Write 3 things you’re grateful for each night <br>• Share a positive moment with someone |

---

### Turning the List into a Sustainable Routine  

1. **Pick 2 habits to start** (e.g., consistent sleep + 10‑min morning walk).  
2. **Habit‑stack**: Pair a new habit with an existing one (water right after brushing).  
3. **Track** with a simple checklist or habit‑tracking app – visual streaks boost motivation.  
4. **Add one habit every 1‑2 weeks** until you have a full set (≈4‑6 weeks total).  
5. **Listen to your body** – modify intensity, timing, or food choices as needed.

---

### Sample “Day‑in‑the‑Life” Blueprint (≈30‑minute read)

| **Time** | **Action** |
|----------|------------|
| **06:30‑07:00** | Wake → glass of water → 10‑min sunlight (walk/balcony) |
| **07:00‑07:30** | Light mobility stretch + 5‑min mindfulness |
| **07:30‑08:00** | Balanced breakfast (e.g., oatmeal + berries + Greek yogurt) |
| **08:00‑12:00** | Work/activities – stand/walk 2 min each hour; sip water |
| **12:00‑12:30** | Lunch (veg‑rich plate + protein) + short outdoor walk |
| **12:30‑13:00** | 5‑min gratitude journal + breathing exercise |
| **13:00‑17:00** | Continue work – repeat hourly 2‑min movement break |
| **17:30‑18:15** | 30‑min moderate cardio (brisk walk, bike, swim) |
| **18:30‑19:00** | Dinner (lean protein, whole grain, veg) + social chat |
| **19:30‑20:00** | Light mental stimulation (reading, puzzle) |
| **20:00‑20:30** | Oral care + screen‑free wind‑down (dim lights, calming music) |
| **21:00** | Bedtime – aim for 7‑9 h sleep |

*(Adjust times to fit your personal schedule; keep the core elements.)*

---

### Bottom Line  

Integrating these **evidence‑based habits**—sleep, hydration, whole‑food nutrition, regular movement, sunlight, stress relief, social ties, mental challenge, hygiene, balanced screen use, posture care, and gratitude—creates a synergistic “health stack.” Start small, build consistency, and the benefits will compound over weeks and months, delivering stronger physical, mental, and emotional wellbeing. 🌱  

**References (selected)**  

1. Sleep Med Rev 2022; 2. Nutrients 2021; 3. Lancet 2023; 4. Cell Metab 2020; 5. JAMA 2022; 6. J Clin Endocrinol Metab 2021; 7. Psychol Bull 2020; 8. PLoS Med 2021; 9. Neurobiol Aging 2022; 10. Circulation 2020; 11. BMJ 2023; 12. Spine 2021; 13. Health Psychol 2020.  

*(Full citation list available on request.)*  

## ⭐ Agent 3: Top 3 Key Points

**Top 3 Science‑Backed Daily Habits for Optimal Health**

| # | Core Habit | Why It Matters (Key Evidence) | How to Make It Stick |
|---|------------|------------------------------|----------------------|
| **1** | **7‑9 h quality sleep each night** | • Enhances cognition, immune function, hormone balance  <br>• Lowers risk of obesity, cardiovascular disease & depression *(Sleep Med Rev 2022)* | • Go to bed & wake at the same time daily (including weekends) <br>• Keep the bedroom cool, dark & quiet (≈18‑20 °C) <br>• Shut off screens ≥ 30 min before bedtime (or use night mode) |
| **2** | **Balanced whole‑food nutrition** (½ plate veg, whole grains, lean protein, healthy fats) | • Reduces heart disease, type‑2 diabetes & certain cancers <br>• Supports a diverse gut microbiome *(Lancet 2023)* | • Fill half the plate with non‑starchy vegetables <br>• Choose oats, quinoa, brown rice for carbs <br>• Add fish, legumes, poultry or tofu for protein <br>• Top meals with nuts, seeds, avocado or olive oil |
| **3** | **≥150 min moderate cardio + 2 strength sessions weekly** (≈30 min most days) | • Improves heart health, insulin sensitivity & mood <br>• Preserves muscle & bone density *(JAMA 2022)* | • Walk, jog, bike, swim or dance 30 min on most days <br>• Add body‑weight, band or weight exercises twice a week <br>• Stand or move for 2 min every 30 min of sitting |

These three habits form the foundation of a health‑optimising routine; once they’re consistent, you can layer the additional habits (hydration, sunlight, stress relief, social connection, etc.) for even greater benefit.

## 📌 Agent 2: Final Refined Summary

## Summary – Top 3 Science‑Backed Daily Habits for Optimal Health  

| # | Core Habit | Why It Matters (Key Evidence) | Practical Tips to Make It Stick |
|---|------------|------------------------------|---------------------------------|
| **1** | **7‑9 h quality sleep each night** | • Boosts cognition, immune function, and hormone balance  <br>• Lowers risk of obesity, cardiovascular disease, and depression *(Sleep Med Rev 2022)* | • Go to bed **and wake up** at the same time every day (yes, weekends). <br>• Keep the bedroom cool (≈18‑20 °C), dark, and quiet. <br>• Power down screens ≥ 30 min before bedtime (or use night‑mode/blue‑light filters). |
| **2** | **Balanced whole‑food nutrition** – ½ plate veg, whole grains, lean protein, healthy fats | • Cuts heart disease, type‑2 diabetes, and some cancers. <br>• Fuels a diverse, resilient gut microbiome *(Lancet 2023)* | • **Plate method:** <br> • Half = non‑starchy vegetables. <br> • Quarter = whole‑grain carbs (oats, quinoa, brown rice). <br> • Quarter = lean protein (fish, legumes, poultry, tofu). <br>• Finish with nuts, seeds, avocado, or olive oil for healthy fats. |
| **3** | **≥150 min moderate cardio + 2 strength sessions per week** (≈30 min most days) | • Improves heart health, insulin sensitivity, and mood. <br>• Preserves muscle and bone density *(JAMA 2022)* | • Choose an activity you enjoy – walk, jog, bike, swim, dance – for ~30 min on most days. <br>• Add a body‑weight, band, or weight routine **twice weekly** (e.g., squats, push‑ups, rows). <br>• Break up long sitting: stand or move for **2 min every 30 min**. |

### Bottom Line  
- **Sleep, nutrition, and regular movement** are the foundational pillars of optimal health.  
- Consistency is built by **simple, repeatable routines** (fixed bedtime, plate‑method meals, short daily activity bursts).  
- Once these three habits are solid, you can layer additional practices (hydration, sunlight, stress‑relief, social connection) for extra health gains.

## 📧 Agent 4: Email Generated from Final Summary

**Subject:** Top 3 Science‑Backed Daily Habits for Optimal Health  

---

Dear [Recipient’s Name],

I’m pleased to share a concise summary of the three daily habits that research consistently links to optimal health. The table below captures each habit, the key evidence supporting it, and practical tips to help you turn them into lasting routines.

| # | Core Habit | Why It Matters (Key Evidence) | Practical Tips to Make It Stick |
|---|------------|------------------------------|---------------------------------|
| **1** | **7‑9 h quality sleep each night** | • Boosts cognition, immune function, and hormone balance  <br>• Lowers risk of obesity, cardiovascular disease, and depression *(Sleep Med Rev 2022)* | • Go to bed **and wake up** at the same time every day (including weekends). <br>• Keep the bedroom cool (≈18‑20 °C), dark, and quiet. <br>• Power down screens ≥ 30 min before bedtime (or use night‑mode/blue‑light filters). |
| **2** | **Balanced whole‑food nutrition** – ½ plate veg, whole grains, lean protein, healthy fats | • Cuts heart disease, type‑2 diabetes, and some cancers. <br>• Fuels a diverse, resilient gut microbiome *(Lancet 2023)* | • **Plate method:** <br> • Half = non‑starchy vegetables. <br> • Quarter = whole‑grain carbs (oats, quinoa, brown rice). <br> • Quarter = lean protein (fish, legumes, poultry, tofu). <br>• Finish with nuts, seeds, avocado, or olive oil for healthy fats. |
| **3** | **≥150 min moderate cardio + 2 strength sessions per week** (≈30 min most days) | • Improves heart health, insulin sensitivity, and mood. <br>• Preserves muscle and bone density *(JAMA 2022)* | • Choose an activity you enjoy – walk, jog, bike, swim, dance – for ~30 min on most days. <br>• Add a body‑weight, band, or weight routine **twice weekly** (e.g., squats, push‑ups, rows). <br>• Break up long sitting: stand or move for **2 min every 30 min**. |

**Bottom Line**  
- **Sleep, nutrition, and regular movement** form the foundational pillars of optimal health.  
- Consistency is built by **simple, repeatable routines** (fixed bedtime, plate‑method meals, short daily activity bursts).  
- Once these three habits are solid, you can layer additional practices (hydration, sunlight, stress‑relief, social connection) for extra health gains.

Please feel free to reach out if you’d like any further details or assistance in implementing these habits.

Best regards,  
[Your Name]  
[Your Title / Position]  
[Your Contact Information]  

In [22]:
from IPython.display import display, Markdown

# -------------------------
# Agent 1
# -------------------------
display(Markdown("### ⏳ Agent 1 executing..."))

health_routines = health_agent.run(
    "Tell me the top most healthy daily routines."
).content

display(Markdown("### ✅ Agent 1 task completed"))
display(Markdown("## 🥗 Agent 1: Healthy Daily Routines"))
display(Markdown(health_routines))


# -------------------------
# Agent 2 (Summary)
# -------------------------
display(Markdown("### ⏳ Agent 2 executing (Summary)..."))

summary_1 = summary_agent.run(health_routines).content

display(Markdown("### ✅ Agent 2 task completed"))
display(Markdown("## ✍️ Agent 2: Summary of Healthy Routines"))
display(Markdown(summary_1))


# -------------------------
# Agent 3 (Top 3)
# -------------------------
display(Markdown("### ⏳ Agent 3 executing (Top 3 extraction)..."))

top_3 = top3_agent.run(summary_1).content

display(Markdown("### ✅ Agent 3 task completed"))
display(Markdown("## ⭐ Agent 3: Top 3 Key Points"))
display(Markdown(top_3))


# -------------------------
# Agent 2 (Final Summary)
# -------------------------
display(Markdown("### ⏳ Agent 2 executing (Final refinement)..."))

final_summary = summary_agent.run(top_3).content

display(Markdown("### ✅ Agent 2 task completed"))
display(Markdown("## 📌 Agent 2: Final Refined Summary"))
display(Markdown(final_summary))


# -------------------------
# Agent 4 (Email)
# -------------------------
display(Markdown("### ⏳ Agent 4 executing (Email generation)..."))

email_content = email_agent.run(final_summary).content

display(Markdown("### ✅ Agent 4 task completed"))
display(Markdown("## 📧 Agent 4: Email Generated from Final Summary"))
display(Markdown(email_content))


### ⏳ Agent 1 executing...

### ✅ Agent 1 task completed

## 🥗 Agent 1: Healthy Daily Routines

## The Top Science‑Backed Daily Routines for Optimal Health  

Below is a concise “starter kit” of daily habits that consistently show strong benefits across multiple peer‑reviewed studies.  Each routine includes **what to do**, **how much**, **when it’s most effective**, and a brief note on the scientific evidence.

| # | Routine | What to Do | Recommended Dose | Best Timing | Why It Works (Key Evidence) |
|---|---------|------------|------------------|-------------|-----------------------------|
| **1** | **Prioritize High‑Quality Sleep** | Keep a regular bedtime/wake‑time, dark cool room, limit screens 30 min before bed. | 7‑9 hours (adults) | 10 pm‑6 am (align with circadian rhythm) | Sleep loss raises cortisol, impairs glucose tolerance & memory. Meta‑analyses link 7‑8 h sleep to ~15 % lower mortality (Cappuccio et al., 2018). |
| **2** | **Move Your Body** – Moderate‑Intensity Aerobic + Strength | • 30 min brisk walk, cycling, swimming, or jog.<br>• 2‑3 sessions/week of resistance (bodyweight, bands, weights). | 150 min/week aerobic + 2‑3 strength sessions | Split aerobic into 10‑15 min bouts if needed; strength any time of day. | Aerobic activity reduces CVD risk by ~30 % (WHO, 2020). Resistance training preserves lean mass & bone density, lowering frailty risk (Rizzoli et al., 2014). |
| **3** | **Hydrate Adequately** | Drink water throughout the day; include herbal teas or low‑sugar fluids. | ≈2.7 L (women) / 3.7 L (men) total fluids, ~½ from plain water. | Start day with 250 ml water; sip regularly; stop ~1 h before sleep. | Mild dehydration impairs cognition, mood, and kidney function (Popkin et al., 2010). |
| **4** | **Eat a Plant‑Rich, Balanced Diet** | • ½ plate vegetables & fruit<br>• Whole grains<br>• Lean protein (fish, legumes, poultry)<br>• Healthy fats (nuts, olive oil)<br>• Limit added sugars, processed meats, sodium. | 5‑7 servings veg/fruit; 3–4 servings whole grains; 2 servings protein daily. | Distribute calories across 3‑5 meals; include a protein source each meal to stabilize blood sugar. | The Mediterranean/DASH patterns cut risk of heart disease & cancer by ~20‑30 % (Estruch et al., 2018; Appel et al., 2011). |
| **5** | **Get Morning Sunlight (Vitamin D & Circadian Reset)** | Spend 10‑30 min outdoors (eyes exposed) soon after waking. | 10‑30 min of bright natural light (≥10 000 lux). | Within 30 min of waking, preferably before 10 am. | Sunlight synchronizes the suprachiasmatic nucleus, improving sleep quality and mood; modest UVB exposure raises serum 25‑OH‑D, linked to bone health and immune function (Holick et al., 2020). |
| **6** | **Practice Stress‑Reduction & Mindfulness** | • 5‑20 min meditation, deep‑breathing, or progressive muscle relaxation.<br>• Journaling or gratitude list. | 10‑20 min/day (can be split). | Morning or early evening; consistent daily cue works best. | Meta‑analysis of mindfulness shows ↓ anxiety, ↓ blood pressure, ↑ immune markers (Goyal et al., 2014). |
| **7** | **Maintain Good Posture & Movement Breaks** | Stand, stretch, or walk 2‑3 min every hour; keep ergonomic workstation. | 5‑10 min per hour of light activity. | Throughout the workday (e.g., Pomodoro 25‑min work/5‑min stretch). | Prolonged sitting raises risk of metabolic syndrome by ~20 % (Owen et al., 2010). Frequent micro‑breaks improve circulation & musculoskeletal health. |
| **8** | **Social Connection & Positive Relationships** | • Call, video chat, or meet a friend/family member.<br>• Participate in community/group activity. | ≥30 min of meaningful interaction. | Anytime that fits your schedule; evening tends to be socially rich. | Loneliness increases mortality risk comparable to smoking (Holt‑Lunstad et al., 2015). Positive social ties buffer stress and support healthy behaviors. |
| **9** | **Limit Alcohol & Avoid Tobacco** | • If you drink, keep ≤1 drink/day (women) or ≤2 (men).<br>• No smoking or vaping. | ≤14 g ethanol (women) / ≤28 g (men) per day. | Alcohol with meals can blunt glucose spikes; avoid binge‑drinking. | Alcohol > moderate raises cancer & CVD risk; smoking is the leading preventable cause of death (CDC, 2022). |
| **10** | **End the Day with a Wind‑Down Routine** | Dim lights, avoid screens, light stretch or reading, set next‑day goals. | 30‑60 min before bedtime. | 1 hour before sleep. | Reduces sympathetic arousal, improves sleep onset latency (Harvard Sleep Health, 2023). |

---

### How to Implement the Routine Seamlessly  

1. **Create a “Daily Blueprint”**  
   - Write a simple schedule (e.g., 6:30 am – wake, 6:45 am – water + sunlight, 7:00 am – walk, 7:30 am – breakfast, 12:00 pm – lunch + brief walk, 5:30 pm – strength training, 9:00 pm – wind‑down).  
   - Use phone reminders or a habit‑tracking app to lock in the cues.

2. **Start Small – The “One‑New‑Habit” Rule**  
   - Pick *one* routine you’re comfortable adding each week.  
   - Once it feels automatic (≈21 days), layer the next habit.

3. **Batch Similar Activities**  
   - Combine sunlight, hydration, and a brief walk first thing in the morning.  
   - Pair strength training with a post‑work shower to streamline timing.

4. **Leverage Social Accountability**  
   - Join a walking group, online meditation class, or strength‑training buddy system.  
   - Share your daily checklist with a friend for mutual encouragement.

5. **Track Progress & Adjust**  
   - Record sleep duration, steps, water intake, mood, and any symptoms.  
   - Review weekly; tweak timing or duration based on energy levels and real‑world constraints.

---

### Quick Reference Cheat‑Sheet  

| Routine | Daily Target | Simple Cue |
|---------|--------------|-----------|
| Sleep | 7‑9 h | Same bedtime & alarm |
| Move | 30 min walk + 2 strength sets | “Walk after coffee” |
| Hydration | 2 L water | “Drink a glass before each meal & snack” |
| Food | Plant‑heavy plate | “Half veg, quarter protein, quarter whole grain” |
| Sunlight | 10‑30 min | “Sit by the window with coffee” |
| Mindfulness | 10 min | “5‑min breathing before lunch” |
| Posture | 5‑10 min breaks/hr | “Stand at the 25‑minute mark” |
| Social | 30 min | “Call a friend after dinner” |
| No/Low Alcohol | ≤1/2 drinks | “Alcohol only on weekend evenings” |
| Wind‑down | 30‑60 min | “Dim lights at 9 pm, read 20 min” |

---

#### Bottom Line  
Incorporating **all ten** of these evidence‑based habits creates a synergistic effect: better metabolic health, stronger immune function, sharper cognition, and a longer, happier life. Start with the habit that feels most doable, build consistency, and gradually expand—you’ll soon have a robust, science‑backed daily routine that feels natural rather than forced. 🌿🚶‍♀️🛌  

*References available on request.*

### ⏳ Agent 2 executing (Summary)...

...

## Quick Summary: 3 Science‑Backed Daily Habits for Optimal Health  

| # | Core Habit | What to Do (Key Action) | Daily Dose & Timing | Main Evidence‑Based Benefit |
|---|------------|------------------------|---------------------|------------------------------|
| **1** | **High‑Quality Sleep** | Fixed bedtime/wake‑time; dark, cool bedroom; no screens ≥ 30 min before sleep | **7‑9 h** (adults), ideally **10 pm – 6 am** to match circadian rhythm | Lowers cortisol, improves glucose tolerance & memory; ~15 % lower all‑cause mortality (Cappuccio et al., 2018) |
| **2** | **Move Your Body** (Aerobic + Strength) | 30 min brisk walk, cycling, swimming **plus** 2‑3 weekly resistance sessions | **≥150 min** moderate aerobic + 2‑3 strength workouts (can be split into 10‑15 min bouts) | Cuts cardiovascular disease risk ≈30 % (WHO 2020); preserves lean mass & bone density (Rizzoli et al., 2014) |
| **3** | **Plant‑Rich Balanced Diet** | Fill half the plate with veg/fruit; add whole grains, lean protein, healthy fats; limit added sugars & processed meats | **5‑7** veg/fruit servings; **3‑4** whole‑grain servings; **2** protein servings across 3‑5 meals | Mediterranean/DASH patterns reduce heart disease & cancer risk 20‑30 % (Estruch et al., 2018; Appel et al., 2011) |

### Why These Three?
- **Foundational pillars**: Sleep, activity, and nutrition most strongly affect hormonal balance, metabolism, immunity, and longevity.  
- **Robust evidence**: Each habit is backed by large epidemiological and interventional studies.  
- **Synergy**: Improving any one habit amplifies the benefits of the others.

### Getting Started
1. **Pick one habit** and implement it consistently for at least two weeks.  
2. **Add the next habit** once the first feels routine.  
3. **Maintain all three** for sustained health gains.  

*Consistency, not perfection, is the key to long‑term well‑being.*

### ⏳ Agent 4 executing (Email generation)...

### ✅ Agent 4 task completed

## 📧 Agent 4: Email Generated from Final Summary

**Subject:** 3 Science‑Backed Daily Habits for Optimal Health  

---

Hi [Name],

I hope you’re doing well. I wanted to share a concise overview of three evidence‑based habits that can markedly improve overall health. Below is a quick reference table followed by a brief guide on how to get started.

---

### Quick Summary: 3 Science‑Backed Daily Habits for Optimal Health  

| # | Core Habit | What to Do (Key Action) | Daily Dose & Timing | Main Evidence‑Based Benefit |
|---|------------|------------------------|---------------------|------------------------------|
| **1** | **High‑Quality Sleep** | Fixed bedtime/wake‑time; dark, cool bedroom; no screens ≥ 30 min before sleep | **7‑9 h** (adults), ideally **10 pm – 6 am** to match circadian rhythm | Lowers cortisol, improves glucose tolerance & memory; ~15 % lower all‑cause mortality (Cappuccio et al., 2018) |
| **2** | **Move Your Body** (Aerobic + Strength) | 30 min brisk walk, cycling, swimming **plus** 2‑3 weekly resistance sessions | **≥150 min** moderate aerobic + 2‑3 strength workouts (can be split into 10‑15 min bouts) | Cuts cardiovascular disease risk ≈30 % (WHO 2020); preserves lean mass & bone density (Rizzoli et al., 2014) |
| **3** | **Plant‑Rich Balanced Diet** | Fill half the plate with veg/fruit; add whole grains, lean protein, healthy fats; limit added sugars & processed meats | **5‑7** veg/fruit servings; **3‑4** whole‑grain servings; **2** protein servings across 3‑5 meals | Mediterranean/DASH patterns reduce heart disease & cancer risk 20‑30 % (Estruch et al., 2018; Appel et al., 2011) |

#### Why These Three?
- **Foundational pillars:** Sleep, activity, and nutrition most strongly affect hormonal balance, metabolism, immunity, and longevity.  
- **Robust evidence:** Each habit is backed by large epidemiological and interventional studies.  
- **Synergy:** Improving any one habit amplifies the benefits of the others.

#### Getting Started
1. **Pick one habit** and implement it consistently for at least two weeks.  
2. **Add the next habit** once the first feels routine.  
3. **Maintain all three** for sustained health gains.  

*Consistency, not perfection, is the key to long‑term well‑being.*

---

Please let me know if you’d like any additional resources or support as you incorporate these habits.

Best regards,  
[Your Name]  
[Your Position]  
[Your Contact Information]  

In [23]:
import os
from dotenv import load_dotenv
from phi.agent import Agent
from phi.model.groq import Groq
from phi.tools.yfinance import YFinanceTools
from phi.tools.email import EmailTools

# Load environment variables from .env file
load_dotenv()

# AGENT 1: Stock Analyst
finance_agent = Agent(
    name="Stock Analyst",
    role="Get financial data",
    model=Groq(id="llama-3.3-70b-versatile"),
    tools=[YFinanceTools(stock_price=True, analyst_recommendations=True)],
    instructions=["Summarize the price and analyst sentiment in a brief table."],
)

# AGENT 2: Summarizer
summarizer_agent = Agent(
    name="News Summarizer",
    role="Summarize latest stock news",
    model=Groq(id="llama-3.3-70b-versatile"),
    instructions=["Provide 3 bullet points of the most important news for this stock today."],
)

# AGENT 3: Email Lead (The Coordinator)
email_agent = Agent(
    name="Email Manager",
    team=[finance_agent, summarizer_agent],
    model=Groq(id="llama-3.3-70b-versatile"),
    tools=[EmailTools(
        receiver_email="recipient@example.com", # Change to your recipient
        sender_email=os.getenv("EMAIL_USER"),
        sender_name="AI Stock Reporter",
        sender_passkey=os.getenv("EMAIL_APP_PASSWORD")
    )],
    instructions=[
        "Get stock data from the Stock Analyst.",
        "Get news from the News Summarizer.",
        "Draft a professional email with a 'Subject: Daily Stock Report - {Ticker}'.",
        "Send the email using the email_user tool."
    ],
    show_tool_calls=True
)

# Run the system
email_agent.print_response("Analyze NVIDIA (NVDA) and email me the report.")

ImportError: `yfinance` not installed. Please install using `pip install yfinance`.

In [ ]:
!pip install yfinance

In [25]:
import os
from dotenv import load_dotenv
from phi.agent import Agent
from phi.model.groq import Groq
from phi.tools.yfinance import YFinanceTools
from phi.tools.email import EmailTools

# Load environment variables from .env file
load_dotenv()

# AGENT 1: Stock Analyst
finance_agent = Agent(
    name="Stock Analyst",
    role="Get financial data",
    model=Groq(id="llama-3.3-70b-versatile"),
    tools=[YFinanceTools(stock_price=True, analyst_recommendations=True)],
    instructions=["Summarize the price and analyst sentiment in a brief table."],
)

# AGENT 2: Summarizer
summarizer_agent = Agent(
    name="News Summarizer",
    role="Summarize latest stock news",
    model=Groq(id="llama-3.3-70b-versatile"),
    instructions=["Provide 3 bullet points of the most important news for this stock today."],
)

# AGENT 3: Email Lead (The Coordinator)
email_agent = Agent(
    name="Email Manager",
    team=[finance_agent, summarizer_agent],
    model=Groq(id="llama-3.3-70b-versatile"),
    tools=[EmailTools(
        receiver_email="janaa7223@gmail.com", # Change to your recipient
        sender_email=os.getenv("EMAIL_USER"),
        sender_name="AI Stock Reporter",
        sender_passkey=os.getenv("EMAIL_APP_PASSWORD")
    )],
    instructions=[
        "Get stock data from the Stock Analyst.",
        "Get news from the News Summarizer.",
        "Draft a professional email with a 'Subject: Daily Stock Report - {Ticker}'.",
        "Send the email using the email_user tool."
    ],
    show_tool_calls=True
)

# Run the system
email_agent.print_response("Analyze NVIDIA (NVDA) and email me the report.")

Output()

INFO     Sending Email to recipient@example.com

KeyboardInterrupt: 

In [27]:
import os
from dotenv import load_dotenv
from phi.agent import Agent
from phi.model.groq import Groq
from phi.tools.yfinance import YFinanceTools
from phi.tools.email import EmailTools

# Load environment variables from .env file
load_dotenv()

# AGENT 1: Stock Analyst
finance_agent = Agent(
    name="Stock Analyst",
    role="Get financial data",
    model=Groq(id="openai/gpt-oss-120b"),
    tools=[YFinanceTools(stock_price=True, analyst_recommendations=True)],
    instructions=["Summarize the price and analyst sentiment in a brief table."],
)

# AGENT 2: Summarizer
summarizer_agent = Agent(
    name="News Summarizer",
    role="Summarize latest stock news",
    model=Groq(id="openai/gpt-oss-120b"),
    instructions=["Provide 3 bullet points of the most important news for this stock today."],
)

# AGENT 3: Email Lead (The Coordinator)
email_agent = Agent(
    name="Email Manager",
    team=[finance_agent, summarizer_agent],
    model=Groq(id="openai/gpt-oss-120b"),
    tools=[EmailTools(
        receiver_email="janaa7223@gmail.com", # Change to your recipient
        sender_email=os.getenv("EMAIL_USER"),
        sender_name="AI Stock Reporter",
        sender_passkey=os.getenv("EMAIL_APP_PASSWORD")
    )],
    instructions=[
        "Get stock data from the Stock Analyst.",
        "Get news from the News Summarizer.",
        "Draft a professional email with a 'Subject: Daily Stock Report - {Ticker}'.",
        "Send the email using the email_user tool."
    ],
    show_tool_calls=True
)

# Run the system
email_agent.print_response("Analyze NVIDIA (NVDA) and email me the report.")

Output()

BadRequestError: Error code: 400 - {'error': {'message': "Tool call validation failed: tool call validation failed: attempted to call tool 'JSON' which was not in request.tools", 'type': 'invalid_request_error', 'code': 'tool_use_failed', 'failed_generation': '{"name": "JSON", "arguments": {\n  "price": 187.38,\n  "currency": "USD",\n  "recommendations": [\n    {\n      "analyst": "Consensus",\n      "rating": "Strong Buy",\n      "count": 11\n    },\n    {\n      "analyst": "Consensus",\n      "rating": "Buy",\n      "count": 49\n    },\n    {\n      "analyst": "Consensus",\n      "rating": "Hold",\n      "count": 3\n    },\n    {\n      "analyst": "Consensus",\n      "rating": "Sell",\n      "count": 1\n    },\n    {\n      "analyst": "Consensus",\n      "rating": "Strong Sell",\n      "count": 0\n    }\n  ]\n}}'}}

In [28]:
import os
from dotenv import load_dotenv
from phi.agent import Agent
from phi.model.groq import Groq
from phi.tools.yfinance import YFinanceTools
from phi.tools.email import EmailTools

# Load environment variables
load_dotenv()

# AGENT 1: Stock Analyst
finance_agent = Agent(
    name="Stock Analyst",
    role="Retrieve financial data",
    # Using the most stable model for tool calling on Groq
    model=Groq(id="llama-3.3-70b-versatile"), 
    tools=[YFinanceTools(stock_price=True, analyst_recommendations=True)],
    instructions=["Retrieve the latest price and analyst ratings. Present them clearly."],
)

# AGENT 2: Summarizer
summarizer_agent = Agent(
    name="News Summarizer",
    role="Summarize latest stock news",
    model=Groq(id="llama-3.3-70b-versatile"),
    instructions=["Find and summarize the top 3 news stories for this stock from today."],
)

# AGENT 3: Email Lead
email_agent = Agent(
    name="Email Manager",
    team=[finance_agent, summarizer_agent],
    model=Groq(id="llama-3.3-70b-versatile"),
    tools=[EmailTools(
        receiver_email="janaa7223@gmail.com",
        sender_email=os.getenv("EMAIL_USER"),
        sender_name="AI Stock Reporter",
        sender_passkey=os.getenv("EMAIL_APP_PASSWORD")
    )],
    instructions=[
        "1. Request data from the Stock Analyst.",
        "2. Request news from the News Summarizer.",
        "3. Combine the info into a report.",
        "4. Use the 'email_user' tool to send the report to janaa7223@gmail.com."
    ],
    show_tool_calls=True
)

# Run the system
email_agent.print_response("Analyze NVIDIA (NVDA) and email me the report.")

Output()

INFO     Sending Email to janaa7223@gmail.com

INFO     Sending Email to janaa7223@gmail.com

In [29]:
import os
from dotenv import load_dotenv
from phi.agent import Agent
from phi.model.groq import Groq
from phi.tools.yfinance import YFinanceTools
from phi.tools.email import EmailTools
from IPython.display import display, Markdown

# Load keys from .env
load_dotenv()

# --- 1. Define Agents ---

# Specialist for raw data
finance_agent = Agent(
    name="Stock Analyst",
    model=Groq(id="llama-3.3-70b-versatile"),
    tools=[YFinanceTools(stock_price=True, analyst_recommendations=True)],
    instructions=["Provide the latest price and analyst ratings for the ticker."],
)

# Specialist for summarizing
summary_agent = Agent(
    name="Summarizer",
    model=Groq(id="llama-3.3-70b-versatile"),
    instructions=["Summarize the provided stock data into 3 concise bullet points."],
)

# Specialist for email drafting
email_agent = Agent(
    name="Email Manager",
    model=Groq(id="llama-3.3-70b-versatile"),
    tools=[EmailTools(
        receiver_email="janaa7223@gmail.com",
        sender_email=os.getenv("EMAIL_USER"),
        sender_name="AI Stock Reporter",
        sender_passkey=os.getenv("EMAIL_APP_PASSWORD")
    )],
    instructions=["Draft a professional email and send it using the 'email_user' tool."],
)

# --- 2. Sequential Execution ---

# Step 1: Get raw stock data
stock_response = finance_agent.run("Analyze NVIDIA (NVDA)")
raw_data = stock_response.content

display(Markdown("## 📊 Agent 1: Raw Stock Analysis"))
display(Markdown(raw_data))


# Step 2: Summarize that data
summary_response = summary_agent.run(raw_data)
final_summary = summary_response.content

display(Markdown("## ✍️ Agent 2: Final Refined Summary"))
display(Markdown(final_summary))


# Step 3: Generate and Send Email
# We pass the final summary to the email agent to act on
email_response = email_agent.run(f"Send this report via email: {final_summary}")

display(Markdown("## 📧 Agent 3: Email Delivery Status"))
display(Markdown(email_response.content))

## 📊 Agent 1: Raw Stock Analysis

In this case, the current price of NVIDIA stock is $187.29. For analyst ratings, out of 64 total ratings, 11 analysts give a strong buy rating, 49 analysts give a buy rating, 3 analysts give a hold rating, 1 analyst gives a sell rating, and 0 analysts give a strong sell rating.

## ✍️ Agent 2: Final Refined Summary

Here are 3 concise bullet points summarizing the provided NVIDIA stock data:

* The current price of NVIDIA stock is $187.29.
* Out of 64 total analyst ratings, 60 analysts recommend buying the stock (11 strong buy, 49 buy).
* Only 4 analysts advise against buying the stock (3 hold, 1 sell), with no strong sell ratings given.

INFO     Sending Email to janaa7223@gmail.com

## 📧 Agent 3: Email Delivery Status

The email has been sent to the user with the subject "NVIDIA Stock Report" and the provided body containing the summary of NVIDIA stock data.

In [30]:
import os
from dotenv import load_dotenv
from phi.agent import Agent
from phi.model.groq import Groq
from phi.tools.yfinance import YFinanceTools
from phi.tools.email import EmailTools
from IPython.display import display, Markdown

# Load environment variables
load_dotenv()

# -------------------------------
# 1. Define Agents
# -------------------------------

# Agent 1: Stock data fetcher
finance_agent = Agent(
    name="Stock Analyst",
    model=Groq(id="llama-3.3-70b-versatile"),
    tools=[YFinanceTools(stock_price=True, analyst_recommendations=True)],
    instructions=[
        "Provide the latest stock price and analyst recommendations clearly."
    ],
)

# Agent 2: Summarizer
summary_agent = Agent(
    name="Summarizer",
    model=Groq(id="llama-3.3-70b-versatile"),
    instructions=[
        "Summarize the stock data into exactly 3 concise bullet points."
    ],
)

# Agent 3: Email Formatter (NEW)
email_formatter_agent = Agent(
    name="Email Formatter",
    model=Groq(id="llama-3.3-70b-versatile"),
    instructions=[
        "Create a professional stock report email.",
        "Include:",
        "- A clear subject line",
        "- A greeting",
        "- The summarized stock insights",
        "- A professional closing",
        "Return the output in this exact format:",
        "Subject: <subject line>\n\nEmail Body:\n<email content>"
    ],
)

# Agent 4: Email Sender (NEW final agent)
email_sender_agent = Agent(
    name="Email Sender",
    model=Groq(id="llama-3.3-70b-versatile"),
    tools=[
        EmailTools(
            receiver_email="janaa7223@gmail.com",
            sender_email=os.getenv("EMAIL_USER"),
            sender_name="AI Stock Reporter",
            sender_passkey=os.getenv("EMAIL_APP_PASSWORD"),
        )
    ],
    instructions=[
        "Send the email exactly as provided using the email_user tool."
    ],
)

# -------------------------------
# 2. Sequential Execution
# -------------------------------

# Step 1: Get raw stock data
stock_response = finance_agent.run("Analyze NVIDIA (NVDA)")
raw_data = stock_response.content

display(Markdown("## 📊 Agent 1: Raw Stock Analysis"))
display(Markdown(raw_data))

# Step 2: Summarize data
summary_response = summary_agent.run(raw_data)
final_summary = summary_response.content

display(Markdown("## ✍️ Agent 2: Summary"))
display(Markdown(final_summary))

# Step 3: Format professional email
formatted_email_response = email_formatter_agent.run(final_summary)
formatted_email = formatted_email_response.content

display(Markdown("## 📨 Agent 3: Formatted Email"))
display(Markdown(formatted_email))

# Step 4: Send email
email_send_response = email_sender_agent.run(formatted_email)

display(Markdown("## ✅ Agent 4: Email Delivery Status"))
display(Markdown(email_send_response.content))


## 📊 Agent 1: Raw Stock Analysis

The current stock price of NVIDIA (NVDA) is $187.21, and the analyst recommendations are as follows:
- Strong Buy: 11
- Buy: 49
- Hold: 3
- Sell: 1
- Strong Sell: 0

## ✍️ Agent 2: Summary

Here are 3 concise bullet points summarizing the stock data:
* The current stock price of NVIDIA (NVDA) is $187.21.
* The majority of analyst recommendations (49) are to "Buy" the stock.
* Overall, analyst sentiment is strongly positive, with 60 out of 64 recommendations being either "Strong Buy" or "Buy".

## 📨 Agent 3: Formatted Email

Subject: NVIDIA (NVDA) Stock Report and Analyst Insights

Email Body:
Dear Investor,

We have compiled the latest data on NVIDIA (NVDA) to provide you with a concise overview of the stock's current standing. The key points are as follows:
* The current stock price of NVIDIA (NVDA) is $187.21.
* The majority of analyst recommendations (49) are to "Buy" the stock.
* Overall, analyst sentiment is strongly positive, with 60 out of 64 recommendations being either "Strong Buy" or "Buy".

We hope this information is helpful in making informed investment decisions. Please do not hesitate to contact us if you require further analysis or have any questions.

Best regards,
[Your Name]

INFO     Sending Email to janaa7223@gmail.com

## ✅ Agent 4: Email Delivery Status

I hope this helps you with your email task. Please let me know if I can be of further assistance!

In [31]:
import os
from dotenv import load_dotenv
from phi.agent import Agent
from phi.model.groq import Groq
from phi.tools.gmail import GmailTools
from IPython.display import display, Markdown

# Load environment variables
load_dotenv()

# --- 1. Define Agents ---

# Agent 1: The Reader
# Specializes in accessing your inbox and retrieving unread messages.
reader_agent = Agent(
    name="Email Reader",
    model=Groq(id="llama-3.3-70b-versatile"),
    tools=[GmailTools()],
    instructions=[
        "Use the 'list_unread_emails' tool to find unread messages.",
        "For each unread email, retrieve the snippet or content.",
        "Return the raw list of emails including sender and subject."
    ],
)

# Agent 2: The Summarizer
# Specializes in taking raw text and making it readable.
summary_agent = Agent(
    name="Summarizer",
    model=Groq(id="llama-3.3-70b-versatile"),
    instructions=[
        "You will receive a list of emails.",
        "Summarize each email into one clear sentence.",
        "Categorize them (e.g., Work, Personal, Newsletter, Action Required).",
        "Use Markdown for a clean output."
    ],
)

# --- 2. Sequential Execution ---

# Step 1: Agent 1 reads all unread emails
display(Markdown("## 🔍 Agent 1: Reading Unread Emails..."))
read_response = reader_agent.run("List all my unread emails and provide their details.")
unread_emails_raw = read_response.content
display(Markdown(unread_emails_raw))

# Step 2: Agent 2 summarizes the retrieved content
display(Markdown("## ✍️ Agent 2: Summarizing Inbox"))
summary_response = summary_agent.run(unread_emails_raw)
final_summary = summary_response.content
display(Markdown(final_summary))

ModuleNotFoundError: No module named 'phi.tools.gmail'

In [32]:
!pip install -U agno google-api-python-client google-auth-oauthlib google-auth-httplib2

  Using cached google_auth_oauthlib-1.2.3-py3-none-any.whl.metadata (3.1 kB)
  Using cached google_auth-2.41.1-py2.py3-none-any.whl.metadata (6.6 kB)
  Using cached h2-4.3.0-py3-none-any.whl.metadata (5.1 kB)
  Using cached hyperframe-6.1.0-py3-none-any.whl.metadata (4.3 kB)
  Using cached hpack-4.1.0-py3-none-any.whl.metadata (4.6 kB)
   ---------------------------------------- 0.0/1.6 MB ? eta -:--:--
   ---------------------------------------- 0.0/1.6 MB ? eta -:--:--
   ---------------------------------------- 0.0/1.6 MB ? eta -:--:--
   ---------------------------------------- 0.0/1.6 MB ? eta -:--:--
   ------ --------------------------------- 0.3/1.6 MB ? eta -:--:--
   ------ --------------------------------- 0.3/1.6 MB ? eta -:--:--
   ------ --------------------------------- 0.3/1.6 MB ? eta -:--:--
   ------------- -------------------------- 0.5/1.6 MB 407.2 kB/s eta 0:00:03
   ------------- -------------------------- 0.5/1.6 MB 407.2 kB/s eta 0:00:03
   ------------------- 

ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
google-genai 1.56.0 requires google-auth[requests]<3.0.0,>=2.45.0, but you have google-auth 2.41.1 which is incompatible.

[notice] A new release of pip is available: 24.3.1 -> 25.3
[notice] To update, run: python.exe -m pip install --upgrade pip
